# Deep Q-Learning 

Reference:
+ [Deep Q-Learning: A Neural Network Approach to Learning Without Experience Replay](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf)
  

## I. Problem Formulation

From the [Problem Formulation](ProblemFormulation_Notation.ipynb), the objective of RL is to maximize the expected returns:
    $$\pi^*= \argmax_\pi J(\pi) \quad \text{where} \quad J(\pi)= \mathbb{E}_{\tau \sim \pi} [R(\tau)]  \quad\quad (1)$$
where $R(\tau) = \sum_{t=0}^T \gamma^t r_t$ is the accumulated returns of a trajectory $\tau=(s_0,a_0,s_1,a_1,...)$. 

Given an initial state $s_0$ or a state-action pair $(s_0,a_0)$, we can define the Value-Function and Q-Function:
+ *Value function*: $V^{\pi}(s) = \mathbb{E}_{\tau \sim \pi} [R(\tau) | s_0=s]$ 
+ *Action-Value function (a.k.a Quality or Q-Function)*: $Q^{\pi}(s,a) = \mathbb{E}_{\tau \sim \pi} [R(\tau) | s_0=s, a_0=a]$ 

Off-Policy methods do not aim to find the policy $\pi$ directly, but instead to build the function $Q(s,a)$. **The motivation is that:** if we know $Q(s,a)$, we can simply pick the action $a=\argmax Q(\cdot|s)$ at the state $s$, and this will achieve our goal of maximizing the expected returns. For example, for an environment with state space $s \in R^n$ and action space $a \in R^m$,
+ **Discrete Action:** If the action is discrete, $a_k=\{0,1\},k=1:m$, we can a build neural network parameterized by $\{\phi\}$ $Q_\phi(\cdot|s): R^n \rightarrow R^m$, and pick the action $a_i$ that has highest $Q$, e.g. $Q(a_i) \geq Q(a_j)$ for $i \neq j$.
+ **Continuous Action:** If the action is continuous, $a_k \in R,k=1:m$, we can a build neural network $Q_\phi(a,s): R^{n+m} \rightarrow R$, and select $a = \argmax Q_\phi(a,s)$ , e.g. by solving $\nabla_a Q(a,s)=0$.  

**Therefore, the problem reduces to how to approximate the function $Q(a,s)$.**


## II. Solution 
### 1.Bellman Equation:
From the definition of accumulated returns $R_t=\sum_{t=0}^\infty \gamma^t r_t$, where $r_t=r(s_t,a_t,s_{t+1})$ is the reward received after acting action $a_t$ at the state $s_t$ and arrive at the new state $s_{t+1}$, we can write it in recursive form:
    $$ R_t = r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + \cdots + \gamma^{T-1} r_{T}  = r_t + \gamma R_{t+1} \quad \quad (2)$$
**Hence, from the definition of Q-function**, we can write it in the recursive form:  
    $$ \boxed{ Q^{\pi}(s_t,a_t)  = \mathbb{E}_{\tau \sim \pi} [R(\tau) | s_0=s_t, a_0=a_t] = \sum_{s_{t+1}} P(s_{t+1}|s_t,a_t) \left[ r(s_t,a_t,s_{t+1}) + \gamma \sum_{a_{t+1}} \pi(a_{t+1}|s_{t+1})Q^{\pi}(s_{t+1},a_{t+1}) \right] }\quad \quad (3)$$
where:
+ The first expectation $\sum_{s_{t+1}} P(s_{t+1}|s_t,a_t) [\cdots ]$ means we may arrive any state $s_{t+1}$ after taking action $a_t$ from the state $s_t$ due to the stochastic of environment, and receive the corresponding reward $r(s_t,a_t,s_{t+1})$.
+ The second expectation $\sum_{a_{t+1}} \pi(a_{t+1}|s_{t+1}) [\cdots ]$ means we can take any action $a_{t+1}$ due to the stochastic of the policy.
+ The last term $Q^{\pi}(s_{t+1},a_{t+1})$ indicates that we need to repeatedly write the first and second expectation for all later actions and states. 

**Similarly, from the definition of Value-function**, we can write it in the recursive form:
    $$ \boxed{ V^{\pi}(s_t)  = \mathbb{E}_{\tau \sim \pi} [R(\tau) | s_0=s_t] = \sum_{a_t} \pi(a_t|s_t) \left[ \sum_{s_{t+1}}  P(s_{t+1}|s_t,a_t) [r(s_t,a_t,s_{t+1}) + \gamma V^{\pi}(s_{t+1})] \right] } \quad \quad (4) $$
where:
+ The first expectation $\sum_{a_t} \pi(a_t|s_t) [\cdots ]$ means we can take any action $a_t$ from the state $s_t$ due to the stochastic of the policy.
+ The second expectation $\sum_{s_{t+1}}  P(s_{t+1}|s_t,a_t) [\cdots ]$ means we can arrive any state $s_{t+1}$ after taking action $a_t$ from the state $s_t$ due to the stochastic of environment. After that, we get the first reward $r(s_t,a_t,s_{t+1})$.
+ The last term $V^{\pi}(s_{t+1})$ indicates that we need to repeatedly write the first and second expectation for all later actions and states. 

**(3) and (4) are called Bellman equation,** and there is the connection between V-Function and Q-Function by their definition:
    $$ V^{\pi}(s_t) = \mathbb{E}_{\tau \sim \pi} [R(\tau) | s_0=s_t] = \sum_{a_t} \pi(a_t|s_t) Q^{\pi}(s_t,a_t) \quad \quad (5)$$
Substituting (5) into (3), we get (6) respectively:
    $$ Q^{\pi}(s_t,a_t) = \sum_{s_{t+1}} P(s_{t+1}|s_t,a_t) \left[ r(s_t,a_t,s_{t+1}) + \gamma V^{\pi}(s_{t+1}) \right] \quad \quad (6)$$

### 2. Optimal Policy:
By definition **the optimal policy is deterministic and greedy**: if there is an action with a maximal Q-value for the optimal policy, it should be systematically taken. For the optimal policy $\pi^*$, the Bellman equations become:
\begin{align*}
    V^*(s_t) &= \max_{a} \sum_{s_{t+1}}  P(s_{t+1}|s_t,a_t) [r_t + \gamma  V^*(s_{t+1})] = \max_a \mathbb{E}_{s_{t+1} \sim P(s_{t+1}|s_t,a_t)} [r_t + \gamma V^*(s_{t+1})]  \quad \quad (7)\\
    Q^*(s_t,a_t) &= \sum_{s_{t+1}}  P(s_{t+1}|s_t,a_t) [r_t + \gamma \max_{a_{t+1}}  Q^*(s_{t+1},a_{t+1})] = \mathbb{E}_{s_{t+1} \sim P(s_{t+1}|s_t,a_t)} [r_t + \gamma \max_{a_{t+1}}  Q^*(s_{t+1},a_{t+1})] \quad \quad (8)\\
\end{align*}

### 3. How to approximate Q-Function:   
From the above analysis, especially the Equ (8), if we ignore the stochastic of the environment, we can get the following property:
    $$Q^*(s_t,a_t) = r_t + \gamma \max_{a_{t+1}}  Q^*(s_{t+1},a_{t+1})$$
Therefore, our approximated $Q_\phi$ must satisfy this property, and we can define the loss function as:
    $$\boxed{L(Q_\phi) = \frac{1}{2}\mathbb{E}_{(s_t \rightarrow a_t \rightarrow s_{t+1})}(Q_{target} - Q_\phi(s_t,a_t))^2 \quad \text{where} \quad Q_{target}= r_t + \gamma \max_{a_{t+1}}  Q_\phi(s_{t+1},a_{t+1}) \quad \quad (9)}$$
Then we can use SGD to update the Q-network: $\phi \leftarrow \phi - \eta \nabla_\phi L(Q_\phi) $.
Note that, $Q_\phi(s,a)$ is just an estimation, which hopefully getting closer and closer to the true value along the learning path. Therefore, after taking an action, there is always a mismatch between the previous and the re-estimated quality, which is called the Temporal Difference.  
    $$ \delta_t = r_t + \gamma \max_{a_{t+1}}  Q_\phi(s_{t+1},a_{t+1}) - Q_\phi(s_t,a_t)$$

### 4. Practical Implementation Issues:
+ **Exploration vs Exploitation Trade-off:** If we strictly follow action that maximizes the Q-value, we only exploit the policy, and there are two issues:
  + Because the Q-function is just an estimation and may not be accurate especially at the initial steps, we may get the wrong action.
  + We don't have the chance to visit the other states and actions that potentially give the higher reward, which leads to sub-optimal learning.
  + **Solution:** Therefore, we always add a small propability $\epsilon$ to take a random action for exploring. This is called $\epsilon$-`greedy policy`. $\epsilon$ should be large at the begining, and gradually decrease to 0 when the our estimation converge. In addition, for discrete action, we can pass Q values to a `softmax` function to generate the propability, and then sample the action in stead of just select the action with highest probability. 
+ **Data Correlation**: If the data in a batch is highly correllated, the information to update the Q-network is not useful. 
  + In practice, data collected in sequent steps is quite redundant. For example, when playing Atari, consecutive frames in a short clip are very similar. 
  + **Solution:** Therefore, to effectively update the Q-network, we build a buffer to store the data collected from many episodes, and then randomly sample them from the buffer to train the network.
+ **Non-stationary Target**: Q-Learning is simply just the regression problem. However, different with the standard supervised learning, the target in here is not fixed but change every steps depends on the parameter $\phi$. 
  + Although we need to update the target toward the true value, updating it too fast leads to unstable training. 
  + **Solution:** Therefore, we use another target network $Q_{\phi'}$ to compute $ Q_{target}= r_t + \gamma \max_{a_{t+1}}  Q_{\phi'}(s_{t+1},a_{t+1}) $,
    and update the target network slower than the main network, such as by:
    + k-Step Delay: update $\phi'_t \leftarrow \phi_{t}$ in every k steps, and fix it for other steps.
    + Exponential Moving Average: $\phi'_t \leftarrow (1-\alpha)\phi'_{t-1} + \alpha \phi_{t}$.

}